In [20]:
import os
from groq import Groq
from IPython.display import display, Markdown
import wikipediaapi
import requests
from bs4 import BeautifulSoup

In [21]:
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [22]:
# Function to fetch Wikipedia page content in HTML format by URL
def fetch_wikipedia_page_by_url(url):
    headers = {
        'User-Agent': 'prompt engineering'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Extract the text content from the HTML, ignoring tables, images, etc.
        paragraphs = soup.find_all('p')
        text_content = '\n\n'.join([para.get_text() for para in paragraphs])
        
        return text_content
    else:
        return None

In [23]:
# URL of the Wikipedia page you want to fetch
url = "https://en.wikipedia.org/wiki/Samsung"

# Fetch the HTML content of the page
samsung_html_content = fetch_wikipedia_page_by_url(url)

# Check if the content was fetched successfully and print it
if samsung_html_content:
    print("Content fetched!")
else:
    print(f"Page at '{url}' not found or could not be fetched.")


Content fetched!


In [24]:
# URL of the Wikipedia page you want to fetch
url = "https://en.wikipedia.org/wiki/Apple_Inc."

# Fetch the HTML content of the page
apple_html_content = fetch_wikipedia_page_by_url(url)

# Check if the content was fetched successfully and print it
if apple_html_content:
    print("Content fetched!")
else:
    print(f"Page at '{url}' not found or could not be fetched.")


Content fetched!


## Few shot prompting

In [25]:
def few_shot_prompt(context):
    return f"""
    Context: 
    ExampleCorp
    Founded in 1995 by John Doe and Jane Smith. Headquarters in New York City. Major products include ExampleWidget and ExampleGadget. Services offered are ExampleConsulting and ExampleSupport. The company operates in the technology industry. Revenue of $500 million with 200 employees.

    Output:
    {{
        "Company": "ExampleCorp",
        "Founded": "1995",
        "Founders": ["John Doe", "Jane Smith"],
        "Headquarters": "New York City",
        "Products": ["ExampleWidget", "ExampleGadget"],
        "Services": ["ExampleConsulting", "ExampleSupport"],
        "Industry": "Technology",
        "Revenue": "$500 million",
        "Employees": "200"
    }}

    Context: 
    TechWorks
    Established in 2000 by Alice Johnson. Located in San Francisco. Known for TechTool and TechDevice. Provides services like TechSupport and TechTraining. Operates in the software industry. Annual revenue is $1 billion with 500 employees.

    Output:
    {{
        "Company": "TechWorks",
        "Founded": "2000",
        "Founders": ["Alice Johnson"],
        "Headquarters": "San Francisco",
        "Products": ["TechTool", "TechDevice"],
        "Services": ["TechSupport", "TechTraining"],
        "Industry": "Software",
        "Revenue": "$1 billion",
        "Employees": "500"
    }}

    Context:
    {context}

    Output:
    """

In [26]:
# Set up the chat completion with context from the Wikipedia article
chat_completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": few_shot_prompt(samsung_html_content[:20000])
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=1,
    stop=None,
    stream=False,
)

    
    
print(chat_completion.choices[0].message.content)

{
    "Company": "Samsung Group",
    "Founded": "1938",
    "Founders": ["Lee Byung-chul"],
    "Headquarters": "Suwon, South Korea",
    "Products": ["Electronics", "Semiconductors", "Construction", "Shipbuilding", "Medical services"],
    "Services": ["Financial services", "Insurance", "Telecommunications"],
    "Industry": "Multinational conglomerate",
    "Revenue": "220 trillion KRW ($172.5 billion) (FY 2009), 280 trillion KRW ($258 billion) (FY 2010)",
    "Employees": "Not specified"
}


In [27]:
# Set up the chat completion with context from the Wikipedia article
chat_completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": few_shot_prompt(apple_html_content[:20000])
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
    
print(chat_completion.choices[0].message.content)

{
    "Company": "Apple Inc.",
    "Founded": "1976",
    "Founders": ["Steve Jobs", "Steve Wozniak", "Ronald Wayne"],
    "Headquarters": "Cupertino, California",
    "Products": ["iPhone", "iPad", "Mac", "Apple Watch", "Vision Pro", "Apple TV", "iOS", "iPadOS", "macOS", "watchOS", "tvOS", "visionOS"],
    "Services": ["Apple Card", "Apple Pay", "iCloud", "Apple Music", "Apple TV+"],
    "Industry": "Technology",
    "Revenue": "$394.3 billion",
    "Employees": "Not specified"
}


#### ^^^ Hallicunated

## Zero shot prompting

In [28]:
def zero_shot_prompt(context):
    return f"""
###
Context: {context}
###

Instruction: 
Extract the major points from the context in JSON format to summarize the given context. Ensure to include the specified fields along with any other pertinent information.
Only output the JSON object.

Example JSON format:
{{
    "Company": "<company name>",
    "Founded": "<found date>",
    "Founders": "<founder name>",
    "Headquarters": <location>,
    "Products": [<product 1>, <product 2>],
    "Services": [<service 1>, <service 2>],
    "Industry": <industry the company belong to>,
    "Revenue": "<revenue>,
    "Employees": <number of employees>,
}}
REMEMBER TO ONLY OUTPUT JSON OBJECT!
"""


In [29]:
# Set up the chat completion with context from the Wikipedia article
chat_completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": zero_shot_prompt(samsung_html_content[:20000])
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
    
print(chat_completion.choices[0].message.content)

{
    "Company": "Samsung Group",
    "Founded": "1938",
    "Founders": "Lee Byung-chul",
    "Headquarters": "Samsung Digital City, Suwon, South Korea",
    "Products": ["Electronics", "Semiconductors", "Construction equipment", "Ships", "Medical equipment"],
    "Services": ["Financial services", "Insurance", "Retail", "Advertising"],
    "Industry": "Multinational conglomerate",
    "Revenue": "220 trillion KRW ($172.5 billion) in FY 2009",
    "Employees": "Not specified",
    "Subsidiaries": ["Samsung Electronics", "Samsung Heavy Industries", "Samsung Engineering", "Samsung Biologics", "Samsung Fire & Marine Insurance", "Samsung Life Insurance"],
    "Notable milestones": ["Entered electronics industry in late 1960s", "Became world's largest producer of memory chips in 1992", "Became world's largest mobile phone maker by unit sales in 2012"]
}


In [30]:
# Set up the chat completion with context from the Wikipedia article
chat_completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": zero_shot_prompt(apple_html_content[:20000])
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
    
print(chat_completion.choices[0].message.content)

{
    "Company": "Apple Inc.",
    "Founded": "April 1, 1976",
    "Founders": ["Steve Jobs", "Steve Wozniak", "Ronald Wayne"],
    "Headquarters": "Cupertino, California, in Silicon Valley",
    "Products": ["iPhone", "iPad", "Mac", "Apple Watch", "Vision Pro", "Apple TV"],
    "Services": ["Apple Card", "Apple Pay", "iCloud", "Apple Music", "Apple TV+"],
    "Industry": "Technology",
    "Revenue": "US$394.3 billion (2022)",
    "Employees": "Not specified",
    "Major Events": [
        "1977: Apple Computer, Inc. was incorporated",
        "1980: Apple went public",
        "1984: Introduced the Macintosh",
        "1985: Steve Jobs left Apple",
        "1996: Apple acquired NeXT",
        "1997: Steve Jobs returned to Apple",
        "1998: Introduced the iMac",
        "1999: Introduced the iBook",
        "2001: Released Mac OS X, opened Apple Store retail locations"
    ]
}


## Chain of Thought

In [31]:
def cot_prompt(context):
    return f"""
Your task is to extract information from the given context as a JSON object in JSON format. Let's think step-by-step.
###
Context: {context}
###

Instruction:
You need to summarize the provided context in JSON format. Follow these steps to ensure accuracy:

1. Understand the Context: Read and comprehend the provided context thoroughly.

2. Extract the following information from the context; "Name", "Founders", "Founded", "Headquarters","Industry", "Products", "Services", "Revenue", "Timeline".

3. All Integer values should be of Integer object type.

4. All date data should be in the format DD-MM-YYYY

4. If any of the details are not available in the context you should display null in the value.

5. Display the extracted information in a JSON format.

6. Review the information to ensure that all relevant details are included and accurately represented.

REMEMBER TO ONLY OUTPUT THE JSON OBJECT!
"""


In [32]:
# Set up the chat completion with context from the Wikipedia article
chat_completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": cot_prompt(samsung_html_content[:20000])
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
    
print(chat_completion.choices[0].message.content)

Here is the extracted information in JSON format:

```
{
  "Name": "Samsung Group",
  "Founders": "Lee Byung-chul",
  "Founded": "1938",
  "Headquarters": "Samsung Digital City, Suwon, South Korea",
  "Industry": "Multinational conglomerate",
  "Products": ["Electronics", "Construction", "Shipbuilding", "Semiconductors", "Telecommunications", "Home appliances", "Medical services"],
  "Services": ["Financial services", "Insurance", "Retail", "Advertising", "Biopharmaceuticals"],
  "Revenue": 258000000000,
  "Timeline": {
    "1938": "Founded by Lee Byung-chul",
    "1947": "Moved head office to Seoul",
    "1954": "Founded Cheil Mojik, a textiles company",
    "1960s": "Entered the electronics industry",
    "1970s": "Entered the construction and shipbuilding industries",
    "1980s": "Acquired the Gumi-based Hanguk Jeonja Tongsin and entered telecommunications hardware",
    "1990s": "Globalized its activities and electronics",
    "2000s": "Opened a development center in Warsaw, Polan

In [33]:
# Set up the chat completion with context from the Wikipedia article
chat_completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": cot_prompt(apple_html_content[:20000])
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
    
print(chat_completion.choices[0].message.content)

Here is the extracted information in JSON format:

```
{
  "Name": "Apple Inc.",
  "Founders": ["Steve Jobs", "Steve Wozniak", "Ronald Wayne"],
  "Founded": "01-04-1976",
  "Headquarters": "Cupertino, California, Silicon Valley",
  "Industry": "Technology",
  "Products": ["iPhone", "iPad", "Mac", "Apple Watch", "Vision Pro", "Apple TV"],
  "Services": ["Apple Card", "Apple Pay", "iCloud", "Apple Music", "Apple TV+"],
  "Revenue": 394300000000,
  "Timeline": {
    "1976": "Apple Computer Company founded",
    "1977": "Apple II introduced",
    "1980": "Apple went public",
    "1984": "Macintosh introduced",
    "1985": "Steve Jobs left Apple",
    "1996": "Apple acquired NeXT",
    "1997": "Steve Jobs returned to Apple",
    "1998": "iMac introduced",
    "1999": "iBook introduced",
    "2001": "Mac OS X released, Apple Store retail locations opened"
  }
}
```


## Prompt Chaining

In [34]:
def first_prompt_chain(context):
    return f"""
Your task is to extract accurate information from the given context.
###
{context}
###
You need to give a detailed summary of the given context.
Let's think step by step.
Step 1: Determine what the context is about.
Step 2: Determine miscellaneous information about the company.
Step 3: Identify the most important parts of the context in detail.
"""

In [35]:
# Set up the chat completion with context from the Wikipedia article
first_prompt_completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": first_prompt_chain(apple_html_content[:20000])
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
    
print(first_prompt_completion.choices[0].message.content)

Let's break down the context step by step.

**Step 1: Determine what the context is about.**
The context is about Apple Inc., an American multinational corporation and technology company.

**Step 2: Determine miscellaneous information about the company.**
Here are some miscellaneous facts about Apple:

* Apple is headquartered in Cupertino, California, in Silicon Valley.
* It is best known for its consumer electronics, software, and services.
* Apple's product lineup includes the iPhone, iPad, Mac, Apple Watch, Vision Pro, and Apple TV, as well as software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS, and services like Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+.
* Apple is one of the Big Five American information technology companies, alongside Alphabet (the parent company of Google), Amazon, Meta (the parent company of Facebook), and Microsoft.

**Step 3: Identify the most important parts of the context in detail.**
Here are the most important parts of the conte

In [36]:
def second_prompt_chain(context):
    return f"""
###
{context}
###
You need to categorize all the related information.
Let's think step by step.
Step 1: Determine what the context is about.
Step 2: Determine how many important informations is present in the context.
Step 3: Quantify the relevant informations as much as possible.
Step 4: Categorize all the important information by the type of info they give.
"""

In [37]:
# Set up the chat completion with context from the Wikipedia article
second_prompt_completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": second_prompt_chain(first_prompt_completion.choices[0].message.content)
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
    
print(second_prompt_completion.choices[0].message.content)

Let's break down the context step by step.

**Step 1: Determine what the context is about.**
The context is about Apple Inc., an American multinational corporation and technology company.

**Step 2: Determine how many important informations are present in the context.**
There are 14 important pieces of information in the context.

**Step 3: Quantify the relevant informations as much as possible.**

1. Founding year: 1976
2. Founders: 3 (Steve Jobs, Steve Wozniak, and Ronald Wayne)
3. Headquarters location: Cupertino, California, in Silicon Valley
4. Product lineup: 7 (iPhone, iPad, Mac, Apple Watch, Vision Pro, Apple TV, and software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS)
5. Services: 7 (Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+)
6. Big Five American information technology companies: 5 (Apple, Alphabet, Amazon, Meta, and Microsoft)
7. Apple I introduction year: 1977
8. Apple II introduction year: 1977
9. IPO year: 1980
10. Macintosh introduction year: 198

In [46]:
def third_prompt_chain(context):
    return f"""
###
{context}
###
You need to categorize all the related information.
Let's think step by step.
Step 1: Determine what the context is about.
Step 2: Identify the categories in the context.
Step 3: Make all the information as compact as possible.
Step 4: Extract the following information; "Name", "Founders", "Founded", "Headquarters","Industry", "Products", "Services", "Revenue", "Timeline".
Step 4: Format all the information according to categories into a JSON object.
"""

In [47]:
# Set up the chat completion with context from the Wikipedia article
third_prompt_completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": third_prompt_chain(second_prompt_completion.choices[0].message.content)
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
    
display(Markdown((third_prompt_completion.choices[0].message.content)))

Let's break down the context step by step.

**Step 1: Determine what the context is about.**
The context is about Apple Inc., an American multinational corporation and technology company.

**Step 2: Identify the categories in the context.**
The categories in the context are:

* Company Information
* Founding and Early History
* Headquarters and Location
* Products and Services
* Industry and Market
* Power Struggle and Departure of Founders
* Post-Jobs Era
* Return of Steve Jobs and Resurgence

**Step 3: Make all the information as compact as possible.**

* Founding year: 1976
* Founders: Steve Jobs, Steve Wozniak, and Ronald Wayne
* Headquarters location: Cupertino, California, in Silicon Valley
* Product lineup: iPhone, iPad, Mac, Apple Watch, Vision Pro, Apple TV, and software like iOS, iPadOS, macOS, watchOS, tvOS, and visionOS
* Services: Apple Card, Apple Pay, iCloud, Apple Music, and Apple TV+
* Industry: Technology
* Timeline:
	+ 1976: Founded
	+ 1977: Apple I and Apple II introduction
	+ 1980: IPO
	+ 1983: John Sculley's hiring
	+ 1984: Macintosh introduction
	+ 1985: Steve Jobs' resignation
	+ 1996: NeXT purchase
	+ 1997: Steve Jobs' return

**Step 4: Extract the following information and format it into a JSON object.**

Here is the extracted information in a JSON object format:

```
{
  "Name": "Apple Inc.",
  "Founders": ["Steve Jobs", "Steve Wozniak", "Ronald Wayne"],
  "Founded": 1976,
  "Headquarters": "Cupertino, California, in Silicon Valley",
  "Industry": "Technology",
  "Products": ["iPhone", "iPad", "Mac", "Apple Watch", "Vision Pro", "Apple TV", "iOS", "iPadOS", "macOS", "watchOS", "tvOS", "visionOS"],
  "Services": ["Apple Card", "Apple Pay", "iCloud", "Apple Music", "Apple TV+"],
  "Revenue": null, // Not provided in the context
  "Timeline": {
    "1976": "Founded",
    "1977": "Apple I and Apple II introduction",
    "1980": "IPO",
    "1983": "John Sculley's hiring",
    "1984": "Macintosh introduction",
    "1985": "Steve Jobs' resignation",
    "1996": "NeXT purchase",
    "1997": "Steve Jobs' return"
  }
}
```

Note: The "Revenue" field is left blank as it is not provided in the context.

In [48]:
# Set up the chat completion with context from the Wikipedia article
first_prompt_completion_samsung = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": first_prompt_chain(samsung_html_content[:20000])
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
# Set up the chat completion with context from the Wikipedia article
second_prompt_completion_samsung = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": second_prompt_chain(first_prompt_completion_samsung.choices[0].message.content)
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
# Set up the chat completion with context from the Wikipedia article
third_prompt_completion_samsung = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "user",
            "content": third_prompt_chain(second_prompt_completion_samsung.choices[0].message.content)
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

    
    
display(Markdown((third_prompt_completion_samsung.choices[0].message.content)))

Let's categorize the related information step by step.

**Step 1: Determine what the context is about.**
The context is about Samsung Group, a South Korean multinational manufacturing conglomerate.

**Step 2: Identify the categories in the context.**
The categories in the context are:

1. History
2. Structure
3. Expansion
4. Controversies
5. Businesses and Subsidiaries
6. Impact on South Korea

**Step 3: Make all the information as compact as possible.**
Here is the compact information:

* Founded in 1938 by Lee Byung-chul
* Headquartered in Samsung Digital City, Suwon, South Korea
* Industry: Multinational conglomerate
* Products/Services: Various industries including construction, consumer electronics, financial services, shipbuilding, and medical services
* Revenue: Equivalent to 22.4% of South Korea's GDP
* Timeline:
	+ 1938: Founded as a trading company
	+ 1960s: Entered electronics industry
	+ 1970s: Entered construction and shipbuilding industries
	+ 1980s: Began investing heavily in research and development
	+ 1987: Separated into five business groups after Lee's death
	+ 2020: World's eighth-highest brand value

**Step 4: Extract and format the information into a JSON object.**
Here is the formatted information in a JSON object:

```
{
  "Name": "Samsung Group",
  "Founders": "Lee Byung-chul",
  "Founded": "1938",
  "Headquarters": "Samsung Digital City, Suwon, South Korea",
  "Industry": "Multinational conglomerate",
  "Products/Services": [
    "Construction",
    "Consumer electronics",
    "Financial services",
    "Shipbuilding",
    "Medical services"
  ],
  "Revenue": "Equivalent to 22.4% of South Korea's GDP",
  "Timeline": {
    "1938": "Founded as a trading company",
    "1960s": "Entered electronics industry",
    "1970s": "Entered construction and shipbuilding industries",
    "1980s": "Began investing heavily in research and development",
    "1987": "Separated into five business groups after Lee's death",
    "2020": "World's eighth-highest brand value"
  }
}
```

Let me know if this is what you were looking for!

## Self-consistency

In [42]:
# Set up the chat completion with context from the Wikipedia article
responses = []
for i in range(5):
    _chat_completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "user",
                "content": zero_shot_prompt(samsung_html_content[:20000])
            }
        ],
        temperature=0,
        max_tokens=1024,
        top_p=0.8,
        stop=None,
        stream=False,
    )
    responses.append(_chat_completion)

In [43]:
for response in responses:
    display(Markdown(response.choices[0].message.content))
    print(50*"=")

{
    "Company": "Samsung Group",
    "Founded": "1938",
    "Founders": "Lee Byung-chul",
    "Headquarters": "Samsung Digital City, Suwon, South Korea",
    "Products": ["Electronics", "Semiconductors", "Construction equipment", "Ships", "Medical equipment"],
    "Services": ["Financial services", "Insurance", "Retail", "Advertising"],
    "Industry": "Multinational conglomerate",
    "Revenue": "220 trillion KRW ($172.5 billion) in FY 2009",
    "Employees": "Not specified",
    "Subsidiaries": ["Samsung Electronics", "Samsung Heavy Industries", "Samsung Engineering", "Samsung Biologics", "Samsung Fire & Marine Insurance", "Samsung Life Insurance"],
    "Notable milestones": ["World's largest mobile phone maker by unit sales in 2012", "World's second-largest chipmaker after Intel", "Largest provider of general insurance in South Korea", "Largest provider of life insurance in South Korea"]
}

{
    "Company": "Samsung Group",
    "Founded": "1938",
    "Founders": "Lee Byung-chul",
    "Headquarters": "Samsung Digital City, Suwon, South Korea",
    "Products": ["Electronics", "Semiconductors", "Construction equipment", "Ships", "Aircraft engines", "Gas turbines"],
    "Services": ["Financial services", "Insurance", "Retail", "Advertising", "Biopharmaceuticals"],
    "Industry": "Conglomerate",
    "Revenue": "220 trillion KRW (approximately $172.5 billion USD) in 2009",
    "Employees": "Not specified",
    "Subsidiaries": ["Samsung Electronics", "Samsung Heavy Industries", "Samsung Engineering", "Samsung Life Insurance", "Samsung Fire & Marine Insurance", "Samsung Biologics", "Samsung Bioepis"],
    "Notable Achievements": ["World's largest mobile phone maker by unit sales in 2012", "World's second-largest chipmaker after Intel", "Largest provider of general insurance in South Korea", "Largest provider of life insurance in South Korea"]
}

{
    "Company": "Samsung Group",
    "Founded": "1938",
    "Founders": "Lee Byung-chul",
    "Headquarters": "Samsung Digital City, Suwon, South Korea",
    "Products": ["Electronics", "Semiconductors", "Construction equipment", "Ships", "Aircraft engines", "Gas turbines"],
    "Services": ["Financial services", "Insurance", "Retail", "Advertising", "Biopharmaceuticals"],
    "Industry": "Conglomerate",
    "Revenue": "220 trillion KRW (approximately $172.5 billion USD) in 2009",
    "Employees": "Not specified",
    "Subsidiaries": ["Samsung Electronics", "Samsung Heavy Industries", "Samsung Engineering", "Samsung Life Insurance", "Samsung Fire & Marine Insurance", "Samsung Biologics", "Samsung Bioepis"],
    "Notable Achievements": ["World's largest mobile phone maker by unit sales in 2012", "World's second-largest chipmaker after Intel", "Largest provider of general insurance in South Korea", "Largest provider of life insurance in South Korea"]
}

{
    "Company": "Samsung Group",
    "Founded": "1938",
    "Founders": "Lee Byung-chul",
    "Headquarters": "Samsung Digital City, Suwon, South Korea",
    "Products": ["Electronics", "Semiconductors", "Construction equipment", "Ships", "Medical equipment"],
    "Services": ["Financial services", "Insurance", "Retail", "Advertising"],
    "Industry": "Multinational conglomerate",
    "Revenue": "220 trillion KRW ($172.5 billion) in FY 2009",
    "Employees": "Not specified",
    "Subsidiaries": ["Samsung Electronics", "Samsung Heavy Industries", "Samsung Engineering", "Samsung Biologics", "Samsung Fire & Marine Insurance", "Samsung Life Insurance"],
    "Notable milestones": ["Entered electronics industry in late 1960s", "Became world's largest producer of memory chips in 1992", "Became world's largest mobile phone maker by unit sales in 2012"]
}

{
    "Company": "Samsung Group",
    "Founded": "1938",
    "Founders": "Lee Byung-chul",
    "Headquarters": "Samsung Digital City, Suwon, South Korea",
    "Products": ["Electronics", "Semiconductors", "Construction equipment", "Ships", "Aircraft engines", "Gas turbines"],
    "Services": ["Financial services", "Insurance", "Retail", "Advertising", "Biopharmaceuticals"],
    "Industry": "Conglomerate",
    "Revenue": "220 trillion KRW (approximately $172.5 billion USD) in 2009",
    "Employees": "Not specified",
    "Subsidiaries": ["Samsung Electronics", "Samsung Heavy Industries", "Samsung Engineering", "Samsung Life Insurance", "Samsung Fire & Marine Insurance", "Samsung Biologics", "Samsung Bioepis"]
}

In [44]:
def self_consistency_prompt(responses):
    # Create a prompt to evaluate consistency
    return f"""
You need to return the most consistent information in JSON format.

You are given a list of several responses to the same question. Your task is to determine which information from all the responses are the most consistent with the others. 

Responses: {responses}

Evaluate all of the information in the responses and determine the most consistent responses.

Then display the most consistent response in JSON format.
"""

In [45]:

consistency_evaluation = client.chat.completions.create(
    model="llama3-70b-8192",  # or your specific model
    messages=[
        {
            "role": "user",
            "content": self_consistency_prompt(responses)
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=0.8,
    stop=None,
    stream=False,
)

# Extract the most consistent response
display(Markdown(consistency_evaluation.choices[0].message.content))

After evaluating the responses, I found that the most consistent information is:

```
{
    "Company": "Samsung Group",
    "Founded": "1938",
    "Founders": "Lee Byung-chul",
    "Headquarters": "Samsung Digital City, Suwon, South Korea",
    "Products": ["Electronics", "Semiconductors", "Construction equipment", "Ships"],
    "Services": ["Financial services", "Insurance", "Retail", "Advertising"],
    "Industry": "Conglomerate",
    "Revenue": "220 trillion KRW (approximately $172.5 billion USD) in 2009",
    "Employees": "Not specified",
    "Subsidiaries": ["Samsung Electronics", "Samsung Heavy Industries", "Samsung Engineering", "Samsung Life Insurance", "Samsung Fire & Marine Insurance", "Samsung Biologics"],
    "Notable Achievements": ["World's largest mobile phone maker by unit sales in 2012", "World's second-largest chipmaker after Intel", "Largest provider of general insurance in South Korea", "Largest provider of life insurance in South Korea"]
}
```

This response is the most consistent because it has the most common information across all the responses, with only minor variations in the "Products" and "Notable Achievements" fields.